In [6]:
import pandas as pd
import os
import pandas_datareader.data as web
import pandas_datareader as pdr
from datetime import datetime
import matplotlib
import yfinance as yf
from datetime import timedelta
yf.pdr_override()
%matplotlib inline
matplotlib.style.use('ggplot')
ntickers = pd.read_csv("nasdaq_ticker_info.txt", sep="\t")

# Hypothesis 1

---

### Firm Ratings actually indicate future price

In [19]:
tickerSymbol = 'MSFT'
#get data on this ticker
tickerData = yf.Ticker(tickerSymbol)
recs = tickerData.recommendations
recs.head()

,Firm,To Grade,From Grade,Action
Date,,,,
2012-03-16,Argus Research,Buy,,up
2012-03-19,Hilliard Lyons,Long-Term Buy,,main
2012-03-22,Morgan Stanley,Overweight,,main
2012-04-03,UBS,Buy,,main
2012-04-20,UBS,Buy,,main


In [20]:
actions = recs['Action'].drop_duplicates().values
print("Actions: ")
print(len(actions))
# print(actions)
print("From Grades: ")
fromgrades = recs['From Grade'].drop_duplicates().values
print(len(fromgrades))
# print(fromgrades)
print("To Grades: ")
togrades = recs['To Grade'].drop_duplicates().values
print(len(togrades))
# print(togrades)
print("Firms: ")
firms = recs['Firm'].drop_duplicates().values
print(len(firms))
# print(firms)

Actions: 
4
From Grades: 
16
To Grades: 
14
Firms: 
53


---

# Hypothesis 2

---

### Comanies with a low close X% (`frac_diff`) below the Y month (`low_window`) high will be up Z days (nxt) from now

In [111]:
goog = return_hdata("GOOG", start_time, end_time)
goog.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-02-09,810.659973,804.539978,809.510010,809.559998,989700,809.559998
2017-02-10,815.250000,809.780029,811.700012,813.669983,1135000,813.669983
2017-02-13,820.958984,815.489990,816.000000,819.239990,1213300,819.239990
2017-02-14,823.000000,816.000000,819.000000,820.450012,1054700,820.450012
2017-02-15,823.000000,818.469971,819.359985,818.979980,1313600,818.979980


In [82]:
calculate_return(100, 130)

0.3

In [38]:
symbol = "AAPL"
start_time = datetime(2017, 2, 9)
end_time = datetime(2017, 5, 24)
from datetime import timedelta

def convert_date(time):
    return str(time).split(" ")[0]

def prevlow_nexthigh(symbol, frac_diff=0.3, max_weeks = 48, appraisal_days=3, sell_days=2):
    now_time = datetime.now()+timedelta(days=-1)
    appraisal_time = now_time+timedelta(days=-(appraisal_days))
    sell_time = now_time+timedelta(days=-sell_days)
    high_time = now_time+timedelta(weeks=-max_weeks)
    high_price = return_hdata(symbol, convert_date(high_time), convert_date(appraisal_time+timedelta(days=-1))).Close.max()
    buy_price = return_hdata(symbol, convert_date(appraisal_time), convert_date(appraisal_time)).Close.max()
    relative_low_close = calculate_return(high_price, buy_price)
    sell_price = return_hdata(symbol, convert_date(sell_time), convert_date(sell_time)).Open.max()
    # calculate return
    pct_return = calculate_return(buy_price, sell_price)
    return pct_return, relative_low_close

def get_stats(tickers):
    newdf = pd.DataFrame()
    for symbol in tickers:
        try:
            result = prevlow_nexthigh(symbol)
            pct_return, relative_low_close = result[0], result[1]
            tdf = pd.DataFrame({'symbol': symbol, 'pct_return': pct_return, 'relative_low_close': relative_low_close}, index=[0])
            mdf = newdf
            newdf = pd.concat([mdf.copy(), tdf])
        except:
            print("failed for: " + symbol)
    return newdf

In [39]:
xtickers = ['GOOG', 'AAPL', 'TSLA', 'PCG', 'NTRA', 'ILMN', 'AMZN','FB']
results = get_stats(xtickers)
# results.to_csv("results_first500stocks.txt", sep="\t")
results.head()

,symbol,pct_return,relative_low_close
0,GOOG,-0.010605,-0.116173
0,AAPL,-0.010206,-0.059749
0,TSLA,-0.013857,-0.137843
0,PCG,-0.018902,-0.536117
0,NTRA,0.006816,0.008160


In [65]:
symbol = "AAPL"

def get_stock_floats(symbole):
    infos = yf.Ticker(symbol).info
    kois = [i for i in infos.keys() if isinstance(infos[i], float)]
    return pd.DataFrame((dict((k, infos[k]) for k in kois if k in infos)), index=[0])

In [66]:
appl_stats = get_stock_floats(symbol)
appl_stats.head()

,previousClose,regularMarketOpen,twoHundredDayAverage,trailingAnnualDividendYield,payoutRatio,regularMarketDayHigh,regularMarketPreviousClose,fiftyDayAverage,trailingAnnualDividendRate,open,...,heldPercentInstitutions,trailingEps,SandP52WeekChange,priceToBook,heldPercentInsiders,shortRatio,earningsQuarterlyGrowth,pegRatio,shortPercentOfFloat,regularMarketPrice
0,309.54,300.35,283.601,0.00995,0.2408,307.9,309.54,281.61237,3.08,300.35,...,0.62115,12.728,0.008263,16.965872,0.00066,0.76,-0.027,2.05,0.007,300.35


In [43]:
ntickers[ntickers.Symbol.isin(xtickers)]

for symbol in xtickers:
    print(symbol)
    #ttemp = yf.Ticker(symbol).info
    

,Symbol,Nasdaq Traded,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
12,AAPL,True,Apple Inc. - Common Stock,Q,Q,False,100.0,False,N,NaN,AAPL,False
391,AMZN,True,"Amazon.com, Inc. - Common Stock",Q,Q,False,100.0,False,N,NaN,AMZN,False
2734,FB,True,"Facebook, Inc. - Class A Common Stock",Q,Q,False,100.0,False,N,NaN,FB,False
3452,GOOG,True,Alphabet Inc. - Class C Capital Stock,Q,Q,False,100.0,False,N,NaN,GOOG,False
4100,ILMN,True,"Illumina, Inc. - Common Stock",Q,Q,False,100.0,False,N,NaN,ILMN,False
5662,NTRA,True,"Natera, Inc. - Common Stock",Q,Q,False,100.0,False,N,NaN,NTRA,False
6024,PCG,True,Pacific Gas & Electric Co. Common Stock,N,,False,100.0,False,NaN,PCG,PCG,False
7957,TSLA,True,"Tesla, Inc. - Common Stock",Q,Q,False,100.0,False,N,NaN,TSLA,False


In [ ]:
## fromgrades = recs['From Grade'].drop_duplicates().values
print(len(fromgrades))
print(fromgrades)